# Skin Cancer Classification (HAM10000)
## Binary Classification: Benign vs Malignant
### Using only HAM10000_images_part_1 with class imbalance handling

## 1. Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torchvision.models as models
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.utils.class_weight import compute_class_weight
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import os
from tqdm import tqdm

print(f"PyTorch Version: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

: 

## 2. Load and Prepare Data

In [ ]:
# Load metadata
metadata_path = "archive/HAM10000_metadata.csv"
metadata = pd.read_csv(metadata_path)

print(f"Total samples in metadata: {len(metadata)}")
print(f"\nClass distribution in original data:")
print(metadata['dx'].value_counts())

# Create binary labels: Malignant (1) vs Benign (0)
# Malignant: mel (melanoma), bcc (basal cell carcinoma), akiec (actinic keratoses)
# Benign: nv (melanocytic nevi), bkl (benign keratosis), df (dermatofibroma), vasc (vascular lesions)
malignant_types = ['mel', 'bcc', 'akiec']
metadata['binary_label'] = metadata['dx'].apply(lambda x: 1 if x in malignant_types else 0)

print(f"\nBinary class distribution:")
print(metadata['binary_label'].value_counts())
print(f"Benign: {(metadata['binary_label'] == 0).sum()}")
print(f"Malignant: {(metadata['binary_label'] == 1).sum()}")

In [ ]:
# Filter only images that exist in part_1
image_dir = "archive/HAM10000_images_part_1"
available_images = set([f.replace('.jpg', '') for f in os.listdir(image_dir) if f.endswith('.jpg')])

print(f"Total images in part_1: {len(available_images)}")

# Filter metadata to only include images from part_1
metadata_part1 = metadata[metadata['image_id'].isin(available_images)].reset_index(drop=True)

print(f"\nFiltered samples: {len(metadata_part1)}")
print(f"\nClass distribution in part_1:")
print(f"Benign: {(metadata_part1['binary_label'] == 0).sum()}")
print(f"Malignant: {(metadata_part1['binary_label'] == 1).sum()}")

# Calculate class imbalance ratio
benign_count = (metadata_part1['binary_label'] == 0).sum()
malignant_count = (metadata_part1['binary_label'] == 1).sum()
imbalance_ratio = benign_count / malignant_count
print(f"\nClass imbalance ratio (Benign/Malignant): {imbalance_ratio:.2f}:1")

In [ ]:
# Split into train and validation sets (stratified)
train_df, val_df = train_test_split(
    metadata_part1, 
    test_size=0.2, 
    random_state=42, 
    stratify=metadata_part1['binary_label']
)

print(f"Training samples: {len(train_df)}")
print(f"  Benign: {(train_df['binary_label'] == 0).sum()}")
print(f"  Malignant: {(train_df['binary_label'] == 1).sum()}")

print(f"\nValidation samples: {len(val_df)}")
print(f"  Benign: {(val_df['binary_label'] == 0).sum()}")
print(f"  Malignant: {(val_df['binary_label'] == 1).sum()}")

# Reset indices
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)

## 3. Create Dataset and DataLoaders

In [ ]:
class SkinLesionDataset(Dataset):
    def __init__(self, dataframe, image_dir, transform=None):
        self.df = dataframe
        self.image_dir = image_dir
        self.transform = transform
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image_path = os.path.join(self.image_dir, f"{row['image_id']}.jpg")
        
        # Load image
        image = Image.open(image_path).convert('RGB')
        
        # Apply transforms
        if self.transform:
            image = self.transform(image)
        
        label = row['binary_label']
        
        return image, label

In [ ]:
# Define transforms
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomRotation(20),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

print("Transforms created successfully")

In [ ]:
# Create datasets
train_dataset = SkinLesionDataset(train_df, image_dir, transform=train_transform)
val_dataset = SkinLesionDataset(val_df, image_dir, transform=val_transform)

print(f"Train dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(val_dataset)}")

# Test loading one sample
sample_image, sample_label = train_dataset[0]
print(f"\nSample image shape: {sample_image.shape}")
print(f"Sample label: {sample_label}")

In [ ]:
# Compute class weights for handling imbalance
train_labels = train_df['binary_label'].values
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_labels),
    y=train_labels
)

class_weights = torch.FloatTensor(class_weights).to(device)
print(f"Class weights: Benign={class_weights[0]:.4f}, Malignant={class_weights[1]:.4f}")

# These weights will be used in the loss function

In [ ]:
# Create dataloaders
batch_size = 32

train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=4,
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=4,
    pin_memory=True
)

print(f"Train batches: {len(train_loader)}")
print(f"Validation batches: {len(val_loader)}")

## 4. Define Model

In [ ]:
class SkinCancerClassifier(nn.Module):
    def __init__(self, pretrained=True):
        super(SkinCancerClassifier, self).__init__()
        # Load pretrained ResNet18
        self.resnet = models.resnet18(pretrained=pretrained)
        
        # Replace final layer for binary classification
        num_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(num_features, 1)
        )
    
    def forward(self, x):
        return self.resnet(x)

# Initialize model
model = SkinCancerClassifier(pretrained=True)
model = model.to(device)

print("Model created successfully")
print(f"Model is on device: {next(model.parameters()).device}")

## 5. Training Setup

In [ ]:
# Loss function with class weights to handle imbalance
pos_weight = class_weights[1] / class_weights[0]
criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([pos_weight]).to(device))

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

# Learning rate scheduler
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', patience=3, factor=0.5, verbose=True
)

print(f"Loss function: BCEWithLogitsLoss with pos_weight={pos_weight:.4f}")
print(f"Optimizer: Adam (lr=0.0001)")
print(f"Scheduler: ReduceLROnPlateau")

In [ ]:
def train_epoch(model, loader, criterion, optimizer, device):
    """Train for one epoch"""
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for images, labels in tqdm(loader, desc="Training"):
        images = images.to(device)
        labels = labels.float().to(device).unsqueeze(1)
        
        # Forward pass
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward pass
        loss.backward()
        optimizer.step()
        
        # Track metrics
        running_loss += loss.item() * images.size(0)
        predictions = (torch.sigmoid(outputs) > 0.5).float()
        correct += (predictions == labels).sum().item()
        total += labels.size(0)
    
    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc


def validate_epoch(model, loader, criterion, device):
    """Validate for one epoch"""
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    all_labels = []
    all_preds = []
    all_probs = []
    
    with torch.no_grad():
        for images, labels in tqdm(loader, desc="Validation"):
            images = images.to(device)
            labels = labels.float().to(device).unsqueeze(1)
            
            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            # Track metrics
            running_loss += loss.item() * images.size(0)
            probs = torch.sigmoid(outputs)
            predictions = (probs > 0.5).float()
            correct += (predictions == labels).sum().item()
            total += labels.size(0)
            
            # Store for detailed metrics
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(predictions.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())
    
    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc, all_labels, all_preds, all_probs

print("Training functions defined")

## 6. Train the Model

In [ ]:
# Training configuration
num_epochs = 20
best_val_acc = 0.0

# History tracking
history = {
    'train_loss': [],
    'train_acc': [],
    'val_loss': [],
    'val_acc': []
}

print("Starting training...")
print("=" * 70)

for epoch in range(num_epochs):
    print(f"\nEpoch [{epoch+1}/{num_epochs}]")
    print("-" * 70)
    
    # Train
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
    
    # Validate
    val_loss, val_acc, _, _, _ = validate_epoch(model, val_loader, criterion, device)
    
    # Update learning rate
    scheduler.step(val_loss)
    
    # Store metrics
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)
    
    # Print progress
    print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
    print(f"Val Loss:   {val_loss:.4f} | Val Acc:   {val_acc:.4f}")
    
    # Save best model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), 'best_skin_cancer_model.pth')
        print(f"✓ Best model saved! (Val Acc: {val_acc:.4f})")

print("\n" + "=" * 70)
print(f"Training completed!")
print(f"Best Validation Accuracy: {best_val_acc:.4f}")

## 7. Visualize Training History

In [ ]:
# Plot training history
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Plot loss
ax1.plot(history['train_loss'], label='Train Loss', marker='o')
ax1.plot(history['val_loss'], label='Val Loss', marker='o')
ax1.set_xlabel('Epoch', fontsize=12)
ax1.set_ylabel('Loss', fontsize=12)
ax1.set_title('Training and Validation Loss', fontsize=14, fontweight='bold')
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)

# Plot accuracy
ax2.plot(history['train_acc'], label='Train Accuracy', marker='o')
ax2.plot(history['val_acc'], label='Val Accuracy', marker='o')
ax2.set_xlabel('Epoch', fontsize=12)
ax2.set_ylabel('Accuracy', fontsize=12)
ax2.set_title('Training and Validation Accuracy', fontsize=14, fontweight='bold')
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('training_history.png', dpi=300, bbox_inches='tight')
plt.show()

print("Training history plot saved as 'training_history.png'")

## 8. Evaluate Best Model

In [ ]:
# Load best model
model.load_state_dict(torch.load('best_skin_cancer_model.pth'))
model.eval()

# Get predictions on validation set
val_loss, val_acc, all_labels, all_preds, all_probs = validate_epoch(
    model, val_loader, criterion, device
)

# Convert to numpy arrays
all_labels = np.array(all_labels).flatten()
all_preds = np.array(all_preds).flatten()
all_probs = np.array(all_probs).flatten()

print(f"Final Validation Accuracy: {val_acc:.4f}")
print(f"Final Validation Loss: {val_loss:.4f}")

In [ ]:
# Classification report
print("\nClassification Report:")
print("=" * 70)
print(classification_report(
    all_labels, 
    all_preds, 
    target_names=['Benign', 'Malignant'],
    digits=4
))

In [ ]:
# Confusion matrix
cm = confusion_matrix(all_labels, all_preds)

plt.figure(figsize=(8, 6))
sns.heatmap(
    cm, 
    annot=True, 
    fmt='d', 
    cmap='Blues',
    xticklabels=['Benign', 'Malignant'],
    yticklabels=['Benign', 'Malignant'],
    cbar_kws={'label': 'Count'},
    annot_kws={'size': 14, 'weight': 'bold'}
)
plt.title('Confusion Matrix', fontsize=16, fontweight='bold', pad=20)
plt.ylabel('True Label', fontsize=12)
plt.xlabel('Predicted Label', fontsize=12)
plt.tight_layout()
plt.savefig('confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

print("Confusion matrix saved as 'confusion_matrix.png'")

In [ ]:
# ROC curve and AUC
from sklearn.metrics import roc_curve

auc_score = roc_auc_score(all_labels, all_probs)
fpr, tpr, thresholds = roc_curve(all_labels, all_probs)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, 'b-', linewidth=2, label=f'ROC Curve (AUC = {auc_score:.4f})')
plt.plot([0, 1], [0, 1], 'k--', linewidth=2, label='Random Classifier')
plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title('ROC Curve', fontsize=16, fontweight='bold')
plt.legend(fontsize=11, loc='lower right')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('roc_curve.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"\nAUC Score: {auc_score:.4f}")
print("ROC curve saved as 'roc_curve.png'")

## 9. Final Summary

In [ ]:
print("\n" + "=" * 70)
print("FINAL RESULTS SUMMARY")
print("=" * 70)
print(f"Dataset: HAM10000_images_part_1")
print(f"Total samples: {len(metadata_part1)}")
print(f"Training samples: {len(train_df)}")
print(f"Validation samples: {len(val_df)}")
print(f"\nClass Imbalance Ratio: {imbalance_ratio:.2f}:1 (Benign:Malignant)")
print(f"Imbalance Handling: Class-weighted loss function")
print(f"\nModel: ResNet18 (pretrained)")
print(f"Best Validation Accuracy: {best_val_acc:.4f}")
print(f"AUC Score: {auc_score:.4f}")
print("\nModel saved as: best_skin_cancer_model.pth")
print("=" * 70)